In [7]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [8]:
# fall=r"E:\Fall Detection Dataset\fall_cam0"
nonfall=r"E:\Fall Detection Dataset\nonfall_cam1\nfDatasets"
video_data=[]
labels=[]

In [9]:
for video_folder in sorted(os.listdir(nonfall)):
    video_path = os.path.join(nonfall, video_folder)
    
    if not os.path.isdir(video_path):
        continue  # Skip if it's not a folder
    
    frames = []
    
    # Iterate through frames (images) in sorted order
    for frame in sorted(os.listdir(video_path)):
        frame_path = os.path.join(video_path, frame)
        
        if frame_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(frame_path)  # Read image
            #img = cv2.resize(img, frame_size)  # Resize (optional)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            frames.append(img)
    
    if frames:  # If frames were found
        video_array = np.array(frames)  # Convert list to NumPy array
        video_data.append(video_array)  # Append to dataset
        labels.append(video_folder)  # Store the folder name as label


In [10]:

# Convert to NumPy arrays
#video_data = np.array(video_data, dtype=object)  # Each video has a different number of frames
labels = np.array(labels)

print(f"Total videos loaded: {len(video_data)}")
print(f"Shape of first video: {video_data[0].shape}")

Total videos loaded: 40
Shape of first video: (150, 480, 640, 3)


In [11]:
min_frames=min(video.shape[0] for video in video_data)
min_frames

55

In [12]:
len(video_data)

40

In [13]:
video_data[0] # this is first video

array([[[[ 0,  0,  6],
         [ 6,  5, 14],
         [ 0,  0,  7],
         ...,
         [ 0,  1,  3],
         [ 2,  0,  8],
         [ 4,  0,  6]],

        [[ 0,  2, 10],
         [42, 41, 50],
         [46, 42, 53],
         ...,
         [73, 79, 84],
         [60, 57, 66],
         [ 9,  1, 11]],

        [[ 0,  0,  6],
         [43, 45, 53],
         [48, 42, 53],
         ...,
         [62, 59, 68],
         [63, 58, 67],
         [ 2,  0,  6]],

        ...,

        [[ 4,  0,  3],
         [ 9,  8, 17],
         [ 0,  0, 11],
         ...,
         [ 7,  0,  0],
         [ 5,  0,  0],
         [ 5,  0,  0]],

        [[ 5,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  6],
         ...,
         [ 0,  0,  0],
         [ 0,  1,  1],
         [ 0,  1,  0]],

        [[10,  6,  1],
         [ 2,  0,  0],
         [ 0,  1,  4],
         ...,
         [ 0,  1,  0],
         [ 0,  2,  1],
         [ 0,  2,  1]]],


       [[[ 9,  0,  1],
         [18,  7, 10],
         [ 9, 

In [14]:
video_data[0].shape

(150, 480, 640, 3)

In [15]:
sliced_videos = []
for video in video_data:
    # Calculate the step size
    num_frames = video.shape[0]
    step = num_frames / min_frames  # Compute interval step

    # Select frames at regular intervals
    selected_indices = np.round(np.arange(0, num_frames, step)).astype(int)[:min_frames] # extracts indices
    sliced_video = video[selected_indices] #slices the video
    sliced_videos.append(sliced_video) # adds sliced video to list
        

In [16]:
sliced_videos = np.array(sliced_videos)

In [17]:
sliced_videos.shape

(40, 55, 480, 640, 3)

In [18]:
import tensorflow as tf
# Load MoveNet model
interpreter = tf.lite.Interpreter(model_path=r"E:\jupyter notebook\3.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run MoveNet on an image
def run_movenet(image):
    image_resized = cv2.resize(image, (192, 192))  # Resize to model input
    image_resized = np.expand_dims(image_resized, axis=0).astype(np.float32) / 255.0  # Normalize

    interpreter.set_tensor(input_details[0]['index'], image_resized)
    interpreter.invoke()
    
    keypoints = interpreter.get_tensor(output_details[0]['index'])[0][0][:, :2]  # (17,2)
    return keypoints

# Process video frames
def process_videos(video_frames):
    num_videos, num_frames, _, _, _ = video_frames.shape
    keypoints_data = np.zeros((num_videos, 17 * 2 * num_frames))  # Shape (30, 1870)

    for v in range(num_videos):
        video_keypoints = []
        for f in range(num_frames):
            keypoints = run_movenet(video_frames[v, f])  # (17, 2)
            video_keypoints.append(keypoints.flatten())  # Flatten to (34,)

        keypoints_data[v] = np.concatenate(video_keypoints)  # Flattened shape (1870,)

    return keypoints_data

# Example usage
# video_frames = np.load("path_to_your_numpy_array.npy")  # Load your video frames
# keypoints_output = process_videos(video_frames)  # Output shape (30, 1870)


In [19]:
pro_video=process_videos(sliced_videos)

In [20]:
pro_video.shape

(40, 1870)

In [21]:
data_with_labels = np.column_stack((pro_video,labels))
data_with_labels

array([['0.7104841470718384', '0.15812477469444275',
        '0.5714103579521179', ..., '0.6223034858703613',
        '0.40069442987442017', 'adl-01-cam0-rgb'],
       ['0.710297703742981', '0.1580047309398651', '0.5715748071670532',
        ..., '0.6221573948860168', '0.4004218280315399',
        'adl-02-cam0-rgb'],
       ['0.7106499671936035', '0.15794232487678528',
        '0.5714742541313171', ..., '0.6213392615318298',
        '0.3996724486351013', 'adl-03-cam0-rgb'],
       ...,
       ['0.7137655019760132', '0.16608935594558716',
        '0.6845018267631531', ..., '0.6226952075958252',
        '0.38242313265800476', 'adl-38-cam0-rgb'],
       ['0.7138492465019226', '0.16477090120315552',
        '0.6844746470451355', ..., '0.6197095513343811',
        '0.4000990092754364', 'adl-39-cam0-rgb'],
       ['0.713927149772644', '0.16465473175048828', '0.6845023036003113',
        ..., '0.6196049451828003', '0.39998704195022583',
        'adl-40-cam0-rgb']], dtype='<U32')

In [22]:
np.savetxt("output4.csv", data_with_labels, delimiter=",", fmt="%s")

print("CSV file saved successfully.")

CSV file saved successfully.
